# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [7]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

In [8]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [9]:
from langsmith import Client
client = Client(api_key=os.environ["LANGCHAIN_API_KEY"])
prompts = client.list_prompts(is_public=False)
print(prompts)


repos=[Prompt(repo_handle='pirate', description='', readme='', id='41467aab-ecb7-49c5-afe7-f6ae2b1fc934', tenant_id='7506181e-8a5f-454a-bc4d-ab4006c3ead0', created_at=datetime.datetime(2025, 3, 16, 21, 30, 15, 375075), updated_at=datetime.datetime(2025, 3, 16, 21, 30, 15, 963505), is_public=False, is_archived=False, tags=['ChatPromptTemplate'], original_repo_id=None, upstream_repo_id=None, owner=None, full_name='pirate', num_likes=0, num_downloads=5, num_views=12, liked_by_auth_user=False, last_commit_hash='a1eb3bd490f12bb2cd9547ca8979d174ecea2d6ceaa2d046678ffc98a1c5f6f8', num_commits=1, original_repo_full_name=None, upstream_repo_full_name=None)] total=1


In [17]:
# # Create a LANGSMITH_API_KEY in Settings > API Keys
# from langsmith import Client
# client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("pirate")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [18]:
prompt

ChatPromptTemplate(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate', 'lc_hub_commit_hash': 'a1eb3bd490f12bb2cd9547ca8979d174ecea2d6ceaa2d046678ffc98a1c5f6f8'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a chatbot. speaking only {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [19]:
hydrated_prompt = prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a chatbot. speaking only Spanish', additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you a captain yet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [20]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="llama3.2",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-982', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Sigo explorando, no soy capitán todavía.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1742203531, model='llama3.2', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=14, prompt_tokens=40, total_tokens=54, completion_tokens_details=None, prompt_tokens_details=None))

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [21]:
prompt = client.pull_prompt("pirate", include_model=True)

/home/pi/miniconda3/envs/pocmg/lib/python3.11/json/decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [22]:
prompt

ChatPromptTemplate(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate', 'lc_hub_commit_hash': 'a1eb3bd490f12bb2cd9547ca8979d174ecea2d6ceaa2d046678ffc98a1c5f6f8'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a chatbot. speaking only {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7ab607e68850>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7ab607e909d0>, root_client=<openai.OpenAI object at 0x7ab607e792d0>, root_async_client=<openai.AsyncOpenAI object at 0x7ab607e68f50>, model_name='gpt-4o-mini', temp

Test out your prompt!

In [23]:
prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})

AIMessage(content='¿Eres un capitán ya?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 25, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-41b62590-faec-4a48-bc0c-fc38e8254c11-0', usage_metadata={'input_tokens': 25, 'output_tokens': 9, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [26]:
prompt = client.pull_prompt("pirate:a9e05734")

Run this commit!

In [28]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

hydrated_prompt = prompt.invoke({"question": "What is the world like?", "language": "English"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="llama3.2",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-714', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Yer wonderin' about the state o' the world in this here year 2500, eh? Well, matey, I'll tell ye it's a right fascinatin' place! The seas be filled with hovercraft and airships, carvin' through the waters like giant metal beasts. Me own ship, the Black Swan, runs on pure energy sources, makin' her faster and more stealthy than ever!\n\nThe cities be towering behemoths o' steel and glass, towerin' high into the air like giant skyscrapers. The streets be filled with walking robots, doin' all sorts o' tasks for humanity. People can communicate with one another using neural links in their heads, makin' it easier to keep track o' each other's actions. It be a brave new world indeed!\n\nSpace travel be as common as fishin' trips on the ocean. Folks be livin' and workin' all over the solar system, with colonies on Mars and beyond. The spaceways be filled with 

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [29]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=7506181e-8a5f-454a-bc4d-ab4006c3ead0'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [32]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="llama3.2")

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/6086dbad?organizationId=7506181e-8a5f-454a-bc4d-ab4006c3ead0'